# Day 6: Lanternfish

## Part 1

The sea floor is getting steeper. Maybe the sleigh keys got carried this way?

A massive school of glowing lanternfish swims past. They must spawn quickly to reach such large numbers - maybe exponentially quickly? You should model their growth rate to be sure.

Although you know nothing about this specific species of lanternfish, you make some guesses about their attributes. Surely, each lanternfish creates a new lanternfish once every 7 days.

However, this process isn't necessarily synchronized between every lanternfish - one lanternfish might have 2 days left until it creates another lanternfish, while another might have 4. So, you can model each fish as a single number that represents the number of days until it creates a new lanternfish.

Furthermore, you reason, a new lanternfish would surely need slightly longer before it's capable of producing more lanternfish: two more days for its first cycle.

So, suppose you have a lanternfish with an internal timer value of 3:

*    After one day, its internal timer would become 2.
*    After another day, its internal timer would become 1.
*    After another day, its internal timer would become 0.
*    After another day, its internal timer would reset to 6, and it would create a new lanternfish with an internal timer of 8.
*    After another day, the first lanternfish would have an internal timer of 5, and the second lanternfish would have an internal timer of 7.

A lanternfish that creates a new fish resets its timer to 6, not 7 (because 0 is included as a valid timer value). The new lanternfish starts with an internal timer of 8 and does not start counting down until the next day.

Realizing what you're trying to do, the submarine automatically produces a list of the ages of several hundred nearby lanternfish (your puzzle input). For example, suppose you were given the following list:
```
3,4,3,1,2
```
This list means that the first fish has an internal timer of 3, the second fish has an internal timer of 4, and so on until the fifth fish, which has an internal timer of 2. Simulating these fish over several days would proceed as follows:
```
Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
After  5 days: 5,6,5,3,4,5,6,7,7,8
After  6 days: 4,5,4,2,3,4,5,6,6,7
After  7 days: 3,4,3,1,2,3,4,5,5,6
After  8 days: 2,3,2,0,1,2,3,4,4,5
After  9 days: 1,2,1,6,0,1,2,3,3,4,8
After 10 days: 0,1,0,5,6,0,1,2,2,3,7,8
After 11 days: 6,0,6,4,5,6,0,1,1,2,6,7,8,8,8
After 12 days: 5,6,5,3,4,5,6,0,0,1,5,6,7,7,7,8,8
After 13 days: 4,5,4,2,3,4,5,6,6,0,4,5,6,6,6,7,7,8,8
After 14 days: 3,4,3,1,2,3,4,5,5,6,3,4,5,5,5,6,6,7,7,8
After 15 days: 2,3,2,0,1,2,3,4,4,5,2,3,4,4,4,5,5,6,6,7
After 16 days: 1,2,1,6,0,1,2,3,3,4,1,2,3,3,3,4,4,5,5,6,8
After 17 days: 0,1,0,5,6,0,1,2,2,3,0,1,2,2,2,3,3,4,4,5,7,8
After 18 days: 6,0,6,4,5,6,0,1,1,2,6,0,1,1,1,2,2,3,3,4,6,7,8,8,8,8
```
Each day, a 0 becomes a 6 and adds a new 8 to the end of the list, while each other number decreases by 1 if it was present at the start of the day.

In this example, after 18 days, there are a total of 26 fish. After 80 days, there would be a total of 5934.

Find a way to simulate lanternfish. **How many lanternfish would there be after 80 days?**

In [1]:
import numpy as np
from IPython.display import Markdown

In [2]:
infile = 'input.txt'
ndays = 80
#infile = 'test_input.txt'
#ndays = 18
with open(infile, 'r') as fid:
    # Lanternfish initial condition:
    f0 = [int(x) for x in fid.readline().strip().split(',')]
# Each element of f0 represents a fish and its internal timer value
f0 = np.array(f0, dtype=np.uint64)
#f0

In [3]:
def getfish(ndays):
    # Generate the lanternfish count histogram for initial condition
    hist, bins = np.histogram(f0, bins=range(8))
    #hist, bins = np.histogram([0,1,1,2,3,4,5,6,6,6], bins=range(8))
    # The INDEX of the hist list represents the fish countdown timer
    # value and the VALUE represents the number of fish at that countdown
    # value. So for example if hist = [0, 1, 1, 2, 1, 0, 0], the
    # number of fish at 0 timer value is hist[0]=0, the number of
    # fish at 1 timer value is hist[1]=1 ... etc.
    #hist, bins

    # Each day every fish's timer value ticks down one.
    # This can be represented by simply shifting hist left
    # by one. The fishes at index 2 go to index 1, index 1
    # goes to 0 and index zero wraps around to index 6 as their
    # timer resets

    # Get the number of Lanternfish after ndays
    # Initialize the new fishes array [7count, 8count, NEWFISH]
    histnew = np.zeros(3, dtype=np.uint64)
    for i in range(ndays):
        hist = np.roll(hist, -1)
        # Add the number of 7 count fishes to the number of 6 count fishes
        hist[6] += histnew[0]
        # The number of 8 count fishes become the number of 7 count fishes
        histnew[0] = histnew[1]
        # The number of new fishes become the number of 8 count fishes
        histnew[1] = histnew[2]
        # The number of 0 count fishes becomes the number of new fishes
        histnew[2] = hist[0]
        #print("Day {}: {}, {}".format(i+1, hist, histnew))

    # Sum to get the total number of fishes
    totalfish = hist.sum() + histnew[:2].sum()
    return totalfish

totalfish = getfish(ndays)
#Markdown("After {} days there are **{}** total fishes.".format(ndays, totalfish))

## Part Two

Suppose the lanternfish live forever and have unlimited food and space. Would they take over the entire ocean?

After 256 days in the example above, there would be a total of 26984457539 lanternfish!

**How many lanternfish would there be after 256 days?**

In [4]:
ndays2 = 256
totalfish2 = getfish(ndays2)
#Markdown("After {} days there are **{}** total fishes.".format(ndays2, totalfish2))